### Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE

### Carregando Dados

In [ ]:
# Carregamento de dados dinâmico
import os

data_path = r'D:\Projetos\UFABC\UFABC_SI\Dados'
files = {
    'Benign Traffic.csv': 'Benign',
    'MQTT DDoS Publish Flood.csv': 'DDoS Publish Flood',
    'MQTT DoS Connect Flood.csv': 'DoS Connect Flood',
    'MQTT DoS Publish Flood.csv': 'DoS Publish Flood',
    'MQTT Malformed.csv': 'Malformed'
}

dfs = []
for filename, target_name in files.items():
    try:
        df = pd.read_csv(os.path.join(data_path, filename))
        df['Target'] = target_name
        dfs.append(df)
    except FileNotFoundError:
        print(f"Aviso: Arquivo não encontrado e será ignorado: {filename}")

df_fail = pd.concat(dfs, ignore_index=True)

print("Dados carregados e concatenados com sucesso.")
display(df_fail['Target'].value_counts())


### Pré-processamento e Exploração

In [ ]:
# Conversão de Timestamp e limpeza de nomes de colunas
df_fail['Timestamp'] = pd.to_datetime(df_fail['Timestamp'], errors='coerce')
df_fail.columns = df_fail.columns.str.strip() # Remove espaços em branco dos nomes das colunas

# Remove linhas onde o Timestamp não pôde ser convertido
df_fail.dropna(subset=['Timestamp'], inplace=True)

df_fail.info()

In [ ]:
df_fail.describe()

In [ ]:
# Verificando a distribuição do alvo
sns.countplot(y=df_fail['Target'])
plt.title('Distribuição das Classes de Tráfego')
plt.show()

### Engenharia de Features
# Para análises de série temporal, é crucial que os dados estejam ordenados.

In [ ]:
df_feat = df_fail.sort_values(by=['Flow ID', 'Timestamp']).copy()

# Lista de colunas numéricas para engenharia de features
features_to_engineer = [
    'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
    'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
    'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std',
    'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
    'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
    'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min'
]

# Garantir que todas as colunas existem antes de prosseguir
features_to_engineer = [col for col in features_to_engineer if col in df_feat.columns]

print("Iniciando engenharia de features...")

# 1. Features de Janela Deslizante (Rolling Window)
window_sizes = [5, 10, 20]
for feature in features_to_engineer:
    for window in window_sizes:
        df_feat[f'{feature}_mean_{window}'] = df_feat.groupby('Flow ID')[feature].rolling(window=window, min_periods=1).mean().reset_index(drop=True)
        df_feat[f'{feature}_std_{window}'] = df_feat.groupby('Flow ID')[feature].rolling(window=window, min_periods=1).std().reset_index(drop=True)

# 2. Features de Atraso (Lag Features)
lags = [1, 2, 3]
for feature in features_to_engineer:
    for lag in lags:
        df_feat[f'{feature}_lag_{lag}'] = df_feat.groupby('Flow ID')[feature].shift(lag)

# 3. Features Cíclicas de Tempo
df_feat['hora_sin'] = np.sin(2 * np.pi * df_feat['Timestamp'].dt.hour / 24.0)
df_feat['hora_cos'] = np.cos(2 * np.pi * df_feat['Timestamp'].dt.hour / 24.0)

print("Engenharia de features concluída.")

# Limpeza de NaNs gerados pela engenharia de features
print(f"Tamanho antes da limpeza de NaN: {df_feat.shape}")
df_feat.dropna(inplace=True)
df_feat.reset_index(drop=True, inplace=True)
print(f"Tamanho após a limpeza de NaN: {df_feat.shape}")


### 5: Modelagem (Machine Learning)

#### 5.1. Preparação dos Dados para o Modelo

In [ ]:
# Colunas a serem removidas antes da modelagem
# Inclui identificadores, timestamps, colunas de texto e as features originais que foram transformadas

# Identificar colunas de texto/objeto que não foram tratadas
object_cols = df_feat.select_dtypes(include='object').columns.tolist()
# Manter a coluna 'Target' por enquanto
if 'Target' in object_cols:
    object_cols.remove('Target')

features_to_drop = ['Timestamp', 'Attack Name', 'Flow ID'] + object_cols + features_to_engineer

# Garantir que não estamos tentando dropar colunas que não existem
features_to_drop = list(set([col for col in features_to_drop if col in df_feat.columns]))

X = df_feat.drop(columns=features_to_drop + ['Target'])
y = df_feat['Target']

# Codificar os rótulos (y) de texto para números
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

print(f"Classes encontradas e codificadas: {dict(zip(label_encoder.classes_, range(num_classes)))}")

# Divisão em treino e teste (80/20) de forma temporal
split_index = int(len(X) * 0.8)

X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y_encoded[:split_index], y_encoded[split_index:]
y_train_labels, y_test_labels = y.iloc[:split_index], y.iloc[split_index:] # Guardar labels de texto para relatórios

print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")


#### 5.2. Treinamento do Modelo de Baseline (Random Forest)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1)

print("Iniciando o treinamento do modelo Random Forest...")
rf_model.fit(X_train, y_train)
print("Treinamento concluído.")


#### 5.3. Avaliação do Modelo

In [ ]:
y_pred = rf_model.predict(X_test)

print("--- Relatório de Classificação no Conjunto de Teste ---")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Matriz de Confusão - Random Forest')
plt.ylabel('Verdadeiro')
plt.xlabel('Previsto')
plt.show()


### 8: Abordagem com Deep Learning (LSTM)

#### 8.1. Balanceamento e Normalização

In [ ]:
print("Contagem de classes no treino antes do SMOTE:")
print(y_train_labels.value_counts())

# Aplicando SMOTE apenas nos dados de treino
smote = SMOTE(random_state=42, k_neighbors=1) # k_neighbors=1 pois uma classe tem apenas 2 amostras
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("\nContagem de classes no treino após o SMOTE:")
print(pd.Series(y_train_balanced).value_counts())

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

print("Dados balanceados e normalizados com sucesso.")


#### 8.2. Remodelagem e Construção do Modelo LSTM

In [ ]:
# A LSTM espera uma entrada 3D: (amostras, timesteps, features)
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Construção do Modelo LSTM para classificação multiclasse
model_lstm = Sequential()
model_lstm.add(LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(num_classes, activation='softmax'))

model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_lstm.summary()


#### 8.3. Treinamento e Avaliação do Modelo LSTM

In [ ]:
print("Iniciando o treinamento do modelo LSTM...")
history = model_lstm.fit(X_train_reshaped, y_train_balanced, 
                         epochs=10, 
                         batch_size=64, 
                         validation_data=(X_test_reshaped, y_test), 
                         verbose=1)

# Avaliação
y_pred_proba_lstm = model_lstm.predict(X_test_reshaped)
y_pred_lstm = np.argmax(y_pred_proba_lstm, axis=1)

print("\n--- Relatório de Classificação do Modelo LSTM ---")
print(classification_report(y_test, y_pred_lstm, target_names=label_encoder.classes_))

cm_lstm = confusion_matrix(y_test, y_pred_lstm)
plt.figure(figsize=(10, 7))
sns.heatmap(cm_lstm, annot=True, fmt='d', cmap='Oranges', 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Matriz de Confusão - Modelo LSTM')
plt.ylabel('Verdadeiro')
plt.xlabel('Previsto')
plt.show()
